In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


# Tic Tac Toe Coin

An outer puzzle for an inner [tic-tac-toe puzzle](tic-tac-toe.ipynb). The tic-tac-toe puzzle is the game engine used by the tic-tac-toe coin puzzle to create a new coin with the next state.

## Chialisp
- [tic-tac-toe-coin.clsp](tic-tac-toe-coin.clsp)
- [tic-tac-toe.clsp](tic-tac-toe.clsp)
- [tic-tac-toe.clib](tic-tac-toe.clib)
- [utils.clib](utils.clip)

## Driver Code
- [tic_tac_toe.py](tic_tac_toe.py)
 

In [2]:
from chia.types.blockchain_format.program import Program
from cdv.util.load_clvm import load_clvm
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [3]:
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    curried_tic_tac_toe_puzzle)
print(disassemble(tic_tac_toe_puzzle))
print()
print(disassemble(curried_tic_tac_toe_puzzle))
print()
print(disassemble(curried_tic_tac_toe_puzzle.rest().first().rest()))

(a (q 2 12 (c 2 (c (a 22 (c 2 (c 5 (c 11 (c 23 ()))))) (c 11 ())))) (c (q ((a (i (all (not (= 9 ())) (not (= 21 ())) (not (= 45 ())) (not (= 93 ())) (not (= -67 ())) (not (= 381 ())) (not (= 765 ())) (not (= 1533 ())) (not (= 3069 ()))) (q 1 . -1) (q 2 10 (c 2 (c (c 9 (c 21 (c 45 (c 93 (c -67 (c 381 (c 765 (c 1533 (c 3069 ()))))))))) (c 11 ()))))) 1) 4 (a 8 (c 2 (c 5 (c 11 ())))) (c 5 ())) (a (i (any (a (i (all (= 9 11) (= 21 11) (= 45 11)) (q 1 . 1) ()) 1) (a (i (all (= 93 11) (= -67 11) (= 381 11)) (q 1 . 1) ()) 1) (a (i (all (= 765 11) (= 1533 11) (= 3069 11)) (q 1 . 1) ()) 1) (a (i (all (= 9 11) (= 93 11) (= 765 11)) (q 1 . 1) ()) 1) (a (i (all (= 21 11) (= -67 11) (= 1533 11)) (q 1 . 1) ()) 1) (a (i (all (= 45 11) (= 381 11) (= 3069 11)) (q 1 . 1) ()) 1) (a (i (all (= 9 11) (= -67 11) (= 3069 11)) (q 1 . 1) ()) 1) (a (i (all (= 45 11) (= -67 11) (= 765 11)) (q 1 . 1) ()) 1)) (q . 11) ()) 1) (a (i (any (> () 23) (> 23 (q . 8))) (q 8 (q . "position invalid")) (q 2 30 (c 2 (c 5 (c 11

In [4]:
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash, 
    curried_tic_tac_toe_puzzle)

# player 1 plays position 4
position = 4
print(f'{board} {player} {position}')

solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
next_player = 2
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 0, 0, 0, 0, 0] 1 4
((73 1023) (51 0x573a4741565fd46402d867686094dbae6df8a4c0154941d3e8cffff8e2f9b5df 1023))
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [5]:
player, next_player = next_player, player
position = 5
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 1, 0, 0, 0, 0] 2 5
((73 1023) (51 0x710a171ba6a6aaa0dee9dfbc1b6506ea3a6f1d0a0482f8622d44510564349e40 1023))
   |   |   
---+---+---
   | x | o 
---+---+---
   |   |   


In [6]:
player, next_player = next_player, player
position = 1
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 1, 2, 0, 0, 0] 1 1
((73 1023) (51 0x2a8cbb499a4b6a47ba6fb65330263105137f485413ff409ef195bca8444a1ff7 1023))
   | x |   
---+---+---
   | x | o 
---+---+---
   |   |   


In [7]:
player, next_player = next_player, player
position = 3
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 1, 0, 0, 1, 2, 0, 0, 0] 2 3
((73 1023) (51 0xc647b7044fb67761230fea05d20ce6da8ee9960b4dced8589f78c01063477c9a 1023))
   | x |   
---+---+---
 o | x | o 
---+---+---
   |   |   


In [8]:
player, next_player = next_player, player
position = 7
print(f'{board} {player} {position}')
solution = Program.to([1023, position, next_player])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))

board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
print(board_state)
tic_tac_toe.print_board(board)

[0, 1, 0, 2, 1, 2, 0, 0, 0] 1 7
((51 0x900ddeed -113) (51 0xcafef00d 1023))
1
   | x |   
---+---+---
 o | x | o 
---+---+---
   | x |   


## Simulate Unplayable Board

In [9]:
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    player
)

# player 1 plays position 4
position = 0
print(f'{board} {player} {position}')

next_player = 2
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 0, 0, 0, 0, 0] 1 0
((73 1023) (51 0x23dae9cd96b230c57f487a33cb851d3d348ee08269483f747327b4ae15d33ebe 1023))
 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   


In [10]:
player, next_player = next_player, player
position = 1
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 0, 0, 0, 0, 0, 0, 0, 0] 2 1
((73 1023) (51 0x3f72bdc1b45c7dd01258697f172d2441f9cf3bd9b64fda8784291d9dd4d29816 1023))
 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   


In [11]:
player, next_player = next_player, player
position = 2
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 0, 0, 0, 0, 0, 0, 0] 1 2
((73 1023) (51 0x94977a2c0b27e2090c4ebf8b9c4517678f8f817070893f1fc62191a2c2130c91 1023))
 x | o | x 
---+---+---
   |   |   
---+---+---
   |   |   


In [12]:
player, next_player = next_player, player
position = 3
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 0, 0, 0, 0, 0, 0] 2 3
((73 1023) (51 0xe28939f3536ca8bb5803666a68765ba88b29241558a12b1242740141299887d3 1023))
 x | o | x 
---+---+---
 o |   |   
---+---+---
   |   |   


In [13]:
player, next_player = next_player, player
position = 4
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 0, 0, 0, 0, 0] 1 4
((73 1023) (51 0x1832a6a578a7bd0d6fca069234069be38eb24fc0112ca537473668d220fe358e 1023))
 x | o | x 
---+---+---
 o | x |   
---+---+---
   |   |   


In [14]:
player, next_player = next_player, player
position = 5
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 0, 0, 0, 0] 2 5
((73 1023) (51 0x11ad1a7aecd116ca7e5c66f6f635a83b4ffc48a5a766ed159bff1b4ea12b8a13 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
   |   |   


In [15]:
player, next_player = next_player, player
position = 7
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 2, 0, 0, 0] 1 7
((73 1023) (51 0x37a3704e46115814aedf3078066c37ae6c6049923b7d561c34b32d1be35eb3ce 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
   | x |   


In [16]:
player, next_player = next_player, player
position = 6
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 2, 0, 1, 0] 2 6
((73 1023) (51 0x86006894358afcc8e70a3b6909fb3fd26b8b4eb338873bf86f32266469e9a124 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
 o | x |   


In [17]:
player, next_player = next_player, player
position = 8
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))

[1, 2, 1, 2, 1, 2, 2, 1, 0] 1 8
((51 0x900ddeed -113))


## Get Board From Curried Coin Puzzle

In [18]:
from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

from clvm_tools.binutils import disassemble
board = [0, 1, 1, 0, 2, 0, 2, 0, 0]
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, board, player
)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash, 
    curried_tic_tac_toe_puzzle)

print(disassemble(curried_tic_tac_toe_coin_puzzle))

(a (q 2 (q 2 22 (c 2 (c 5 (c 11 (c 23 (c 431 (c (a 47 (c -65 ())) (c 95 (c (a (i (= 27503 (q . 1)) (q 1 . 2) (q 1 . 1)) 1) ()))))))))) (c (q (73 51 4 (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) ((c (q . 2) (c (c (q . 1) 5) (c (a 26 (c 2 (c 11 (q 1)))) ()))) 2 (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 26 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) (a (i (not (= () -97)) (q 4 (c 20 (q 0x900ddeed -113)) (a (i (= (q . -1) -97) () (q 4 (a (i (= (q . 1) -97) (q 4 20 (c 11 (c -65 ()))) (q 4 20 (c 23 (c -65 ())))) 1) ())) 1)) (q 2 28 (c 2 (c (a 30 (c 2 (c (a 18 (c 2 (c 5 (c (c 5 (c 11 (c 23 (c (a 18 (c 2 (c 47 (c (c 351 (c 383 ())) ())))) ())))) ())))) ()))) (c -65 ()))))) 1) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 2 (q 2 22 (c 2 (c 5 (c 11 (c 23 (c 431 (c (a 47 (c -65 ())) (c 95 (c (a (i (= 27503 (q . 1)) (q 1 . 2) (q 1 . 1)) 1) ()))))))))) (c (q (73 51 4 (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) ((c (q . 2) (c (c (q . 1) 5) 

In [19]:
board_from_puzzle = curried_tic_tac_toe_coin_puzzle.at("rrfrrfrrfrrfrfrrrfrfr")
print(disassemble(board_from_puzzle))

(() 1 1 () 2 () 2 () ())


In [20]:
board_from_puzzle = curried_tic_tac_toe_coin_puzzle.at("rrfrrfrrfrrfrfrrrfrfr")
print(disassemble(board_from_puzzle))
board_from_puzzle = tic_tac_toe.get_board_from_curried_tic_tac_toe_coin_puzzle(curried_tic_tac_toe_coin_puzzle)
tic_tac_toe.print_board(board_from_puzzle)

(() 1 1 () 2 () 2 () ())
   | x | x 
---+---+---
   | o |   
---+---+---
 o |   |   


In [21]:
from clvm.casts import int_to_bytes
board_from_puzzle = curried_tic_tac_toe_coin_puzzle.at("rrfrrfrrfrrfrfrrrfrfr")
solution = Program.to([board_from_puzzle, int_to_bytes(2), int_to_bytes(0)])
result = tic_tac_toe_puzzle.run(solution)
print(disassemble(result))

(() (a 1 1 () 2 () 2 () ()))
